In [39]:
# For Fetching Comments 
from googleapiclient.discovery import build 
# For filtering comments 
import re 
# For filtering comments with just emojis 
import emoji
# Analyze the sentiments of the comment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# For visualization 
import matplotlib.pyplot as plt
import csv
from datetime import datetime
from dotenv import load_dotenv
import os

In [40]:
API_KEY = os.getenv('API_KEY') # Put in your API Key
 
youtube = build('youtube', 'v3', developerKey=API_KEY)

# Taking input from the user and slicing for video id
video_id = input('Enter Youtube Video URL: ')[-11:]
print("video id: " + video_id)

# Getting the channelId of the video uploader
video_response = youtube.videos().list(
    part='snippet',
    id=video_id
).execute()

# Check if 'items' exists and is not empty before accessing it
if 'items' in video_response and video_response['items']:
    video_snippet = video_response['items'][0]['snippet']
    uploader_channel_id = video_snippet['channelId']
    print("channel id: " + uploader_channel_id)
else:
    print("Error: No video found with the given ID. Please check the URL and try again.")

video id: vCwqmlwJbmM
channel id: UCg40OxZ1GYh3u3jBntB6DLg


In [20]:

# Fetch comments
print("Fetching Comments...")
comments = []
nextPageToken = None
while len(comments) < 600:
    request = youtube.commentThreads().list(
        part='snippet',
        videoId=video_id,
        maxResults=30,  # You can fetch up to 100 comments per request
        pageToken=nextPageToken
    )
    response = request.execute()
    for item in response['items']:
        comment = item['snippet']['topLevelComment']['snippet']
        # Check if the comment is not from the video uploader
        if comment['authorChannelId']['value'] != uploader_channel_id:
            comments.append(comment['textDisplay'])
    nextPageToken = response.get('nextPageToken')
 
    if not nextPageToken:
        break
# Print the 5 comments
comments[:25]
print(response['items'][0])

Fetching Comments...
{'kind': 'youtube#commentThread', 'etag': 'xDXgisZojtA3YUbmkmOmmjj1ad4', 'id': 'Ugy0Wb6q9Xkg9qin7Ph4AaABAg', 'snippet': {'channelId': 'UChjd5M1K5Y3vQLgmIveZyzg', 'videoId': 'P0IaOVRL-EM', 'topLevelComment': {'kind': 'youtube#comment', 'etag': 'roGqlFU0pFIUaX_wEvccnyQyChs', 'id': 'Ugy0Wb6q9Xkg9qin7Ph4AaABAg', 'snippet': {'channelId': 'UChjd5M1K5Y3vQLgmIveZyzg', 'videoId': 'P0IaOVRL-EM', 'textDisplay': 'If he wins your story is fake news 😅', 'textOriginal': 'If he wins your story is fake news 😅', 'authorDisplayName': '@barrocjb', 'authorProfileImageUrl': 'https://yt3.ggpht.com/ytc/AIdro_kyeWcCxUUHPTLSxtk1U5ioOwxiuOKEJ9ngl40e4S8=s48-c-k-c0x00ffffff-no-rj', 'authorChannelUrl': 'http://www.youtube.com/@barrocjb', 'authorChannelId': {'value': 'UCarAPHmF1AS6bpD9WHVR_FQ'}, 'canRate': True, 'viewerRating': 'none', 'likeCount': 24, 'publishedAt': '2024-09-12T05:35:15Z', 'updatedAt': '2024-09-12T05:35:15Z'}}, 'canReply': True, 'totalReplyCount': 5, 'isPublic': True}}


video id: qlAO3Ohw3gQ
Fetching Comments...
Total comments fetched: 600
Comments have been saved to youtube_comments_qlAO3Ohw3gQ_20240923_220235.csv


In [44]:

def get_video_details(video_id):
    try:
        video_response = youtube.videos().list(
            part='snippet',
            id=video_id
        ).execute()
        
        if 'items' in video_response and video_response['items']:
            return video_response['items'][0]['snippet']
        else:
            print(f"Error: No video found with ID {video_id}")
            return None
    except HttpError as e:
        print(f"An HTTP error occurred for video ID {video_id}: {str(e)}")
        return None

def get_video_comments(video_id, max_results=100):
    comments = []
    try:
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=max_results
        )
        response = request.execute()

        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments.append({
                'video_id': video_id,
                'text': comment['textDisplay'],
                'likeCount': comment['likeCount'],
                'publishedAt': comment['publishedAt']
            })

        return comments
    except HttpError as e:
        print(f"An HTTP error occurred for video ID {video_id}: {str(e)}")
        return []

def process_videos_from_csv(input_file, output_file):
    all_comments = []

    with open(input_file, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            video_id = row['video_url'].split('v=')[-1].strip()
            video_details = get_video_details(video_id)
            
            if video_details:
                print(f"Fetching comments for video: {video_details['title']}")
                comments = get_video_comments(video_id)
                for comment in comments:
                    comment['video_title'] = video_details['title']
                all_comments.extend(comments)
                print(f"Fetched {len(comments)} comments")
            else:
                print(f"Could not fetch details for video ID: {video_id}")

    if all_comments:
        with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['video_id', 'video_title', 'text', 'likeCount', 'publishedAt']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            writer.writeheader()
            for comment in all_comments:
                writer.writerow(comment)
        
        print(f"Processed comments for {len(set([c['video_id'] for c in all_comments]))} videos. Results saved to {output_file}")
    else:
        print("No comments were fetched.")

# Usage
input_file = 'inp_files/Election 2024 YT Videos - Sheet1.csv'  # Replace with your input CSV file name
output_file = 'inp_files/output_video_comments.csv'  # Replace with your desired output file name

process_videos_from_csv(input_file, output_file)

Error: No video found with ID Alljw46hEwc
Could not fetch details for video ID: Alljw46hEwc
Error: No video found with ID U_-TvkO6bwU
Could not fetch details for video ID: U_-TvkO6bwU
Fetching comments for video: Full Debate: Harris vs. Trump in 2024 ABC News Presidential Debate | WSJ
Fetched 100 comments
Error: No video found with ID OQgE0ETV81s&t=28s
Could not fetch details for video ID: OQgE0ETV81s&t=28s
Fetching comments for video: LIVE: Donald Trump speaks after presidential debate in Arizona
Fetched 100 comments
Fetching comments for video: US Presidential Debate 2024: Top Highlights | Kamala Harris Vs Donald Trump | Best 9 Minutes
Fetched 100 comments
Fetching comments for video: Harris v Trump: highlights of the US presidential election debate
Fetched 100 comments
Fetching comments for video: FULL VIDEO | 2024 Presidential Debate from ABC News
Fetched 100 comments
Fetching comments for video: Donald Trump says no more Kamala Harris debates | BBC News
Fetched 100 comments
Fetchi